In [ ]:
"""
@author Michael Levet
@author Jeremy Alm
@date 08/18/2022

This code tests whether the abstract relation algebra 
\mathcal{A}_{3}([i,i,i]) admits a finite representation 
on n points using a SAT solver. We define a Boolean formula 
\varphi whose satisfiability is a necessary condition to be 
representable on n points. So if \varphi is not satisfiable, 
then \mathcal{A}_{3}([i,i,i]) is not representable on n points.
"""

import boolexpr as bx
import itertools

import cProfile
import signal
from contextlib import contextmanager

In [ ]:
# change the value of n to test if 
# \mathcal{A}_{3}([i,i,i]) is representable on n points
n = 13

vertices = list(range(n))
edges = list(itertools.combinations(vertices, 2))
ctx = bx.Context()
variables = { **{ (i,j,0) : ctx.get_var(str((i,j,0))) for i,j in edges} ,
             **{ (i,j,1) : ctx.get_var(str((i,j,1))) for i,j in edges} ,  
             **{ (i,j,2) : ctx.get_var(str((i,j,2))) for i,j in edges}}


In [ ]:
# exactly one color is true
every_edge_colored = ( ((variables[(i,j,0)] | variables[(i,j,1)] | variables[(i,j,2)])) 
                      for i,j in edges
                     )

conjuncts_colors = ( ~(variables[(i,j,0)] & variables[(i,j,1)]) &
                     ~(variables[(i,j,0)] & variables[(i,j,2)]) &
                     ~(variables[(i,j,1)] & variables[(i,j,2)])    
                    for i,j in edges )

phi_0 = bx.and_s(bx.and_s(*every_edge_colored), bx.and_s(*conjuncts_colors))
print("Made phi_0")


R = {(0,1), (0, 2), (0, 3), (1, 4), (1,5)} 
B = {(1, 2), (0, 4), (0, 6), (1, 6), (0, 7), (1,8)}
G = {(1,3), (0, 5), (1, 7), (0,8), (0,9),(1,9)}

conjuncts_red_subgraph = (variables[(i,j,0)] for i,j in R)
conjuncts_blue_subgraph = (variables[(i,j,1)] for i,j in B)
conjuncts_green_subgraph = (variables[(i,j,2)] for i,j in G)

#color the existing edges by what we know
phi_1 = bx.and_s( bx.and_s(*conjuncts_red_subgraph), bx.and_s(*conjuncts_blue_subgraph),
                bx.and_s(*conjuncts_green_subgraph) )
print("Made phi_1")

def disjunction(i,j,c1,c2):
    # returns a disjunction that says edge ij has need c1-c2 met
    return bx.or_s( *( variables[(min(i,k), max(i,k), c1)] 
                               & variables[(min(j,k), max(j,k), c2)] for k in set(range(n)) - {i,j} ) ) 

# each edge must have its "needs" met


edge_needs_r = {(0,1),(1,0),(0,2),(2,0),(1,1),(2,2),(1,2),(2,1)}

# all possible needs for blue edges
edge_needs_bl = {(1,0),(0,1),(2,1),(1,2),(0,0),(2,2),(2,0),(0,2)}

# all possible needs for green edges
edge_needs_gr = {(2,0),(0,2),(1,2),(2,1),(0,0),(1,1),(0,1),(1,0)}


print("Made phi_2")
conjuncts_forbidden_triangles = ( ~(variables[(i,j,0)] & variables[(i,k,0)] & variables[(j,k,0)]) & 
                                  ~(variables[(i,j,1)] & variables[(i,k,1)] & variables[(j,k,1)]) &
                                  ~(variables[(i,j,2)] & variables[(i,k,2)] & variables[(j,k,2)])
                                 for i,j,k in itertools.combinations(vertices,3))

#forbid all blue-green triangles except for ggg
phi_3 = bx.and_s(*conjuncts_forbidden_triangles)
print("Made phi_3")

#every edge must have its "needs" met
conjuncts_needs_red = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_r ))  
                       for i,j in R)

#every red edge must have its "needs" met
phi_2 = bx.and_s(*conjuncts_needs_red)


#every edge must have its "needs" met
conjuncts_needs_Bl = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_bl ))  
                       for i,j in B)


#every blue edge must have its "needs" met
phi_4 = bx.and_s(*conjuncts_needs_Bl)
print("Made phi_4")

conjuncts_needs_Green = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_gr ))  
                       for i,j in G)


#every green edge must have its "needs" met
phi_5 = bx.and_s(*conjuncts_needs_Green)
print("Made phi_5")





# every edge not colored must have its "needs" met
edge_unknown = set(set(edges) - set(R) - set(B) - set(G))

def needs_r(i,j):
    # returns a disjunction that says edge ij is red and has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_r ))  

def needs_b(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_bl )) 

def needs_g(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_gr )) 


conjuncts_needs_unknown =  (  (variables[(i,j,0)] & needs_r(i,j) ) |  
                                (variables[(i,j,1)] & needs_b(i,j) ) |  
                                (variables[(i,j,2)] & needs_g(i,j) ) 
                              for i,j in edge_unknown)

phi_6 = bx.and_s(*conjuncts_needs_unknown)
print("Made phi_6 \n")



# phi = bx.and_(phi_0, phi_1, phi_2)
phi = bx.and_s(phi_0, phi_1, phi_2, phi_4, phi_5, phi_6)

print("Starting to simplify")
phi_simpl = phi.simplify()
print("Simplified")
phi_tseytin = phi.tseytin(ctx)
print("Tseytin Call Completed")

#print(phi_simpl)
#print(phi)
#phi_tseytin.sat()

#phi_simpl.sat()

#list(phi_simpl.iter_sat())
print("Call SAT")
phi_tseytin.sat()